<a href="https://colab.research.google.com/github/arjadre/balloon_activity/blob/main/BalloonActivityJanuary2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Preliminary stuff:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mplcolors
import urllib.request

# Defining a customized color: [Red, Green, Blue], each from 0 to 1.
# We're going to use this color later on to plot relative humidity.
skyblue_RGB = [0.0, 0.4, 0.9]
skyblue_hex = mplcolors.to_hex(skyblue_RGB)

In [ ]:
# Saturation vapor pressure:
def e_sat(T):
    # Calculates saturation vapor pressure, in hPa, given a temperature or temperature array in °C
    # This formula is not exact, but it is close enough for our purposes today.
    e_sat_vals = 6.112 * np.exp(17.67*T/(243.5 + T))
    return e_sat_vals

# Relative humidity:
def rel_hum(T,T_d):
    # Calculates relative humidity, in percent
    # Inputs:
    #   T = Temperature, in °C
    #   T_d = Dew point, in °C
    p_vap_actual = e_sat(T_d) # the partial pressure of water vapor
    p_vap_saturation = e_sat(T) # the saturation partial pressure of water vapor
    rh_value = 100 * p_vap_actual/p_vap_saturation
    return rh_value

In [ ]:
# Sounding info: (January 6, 2025 at 12 UTC at the OKX site, which is here in Upton, NY)
sounding_year = "25"
sounding_month = "01"
sounding_day = "06"
sounding_hour = "12"
sounding_site = "OKX"
sounding_site_name = "Upton, New York"

# List of sites: https://www.weather.gov/upperair/sounding
# (usually need to remove the K at the beginning: KOKX on the map = OKX in the code above)
# (there are some exceptions, such as TJSJ = San Juan, PR; do not remove the T)
# (data from some sites outside the continental US are not available here)
# Another website with similar info: https://www.spc.noaa.gov/exper/soundings/

# URL of the data file
# This will look like: "https://www.spc.noaa.gov/exper/soundings/25010612_OBS/OKX.txt"
base_url = "https://www.spc.noaa.gov/exper/soundings/"
url = base_url + sounding_year + sounding_month + sounding_day + sounding_hour + "_OBS/" + sounding_site + ".txt"
plot_title = sounding_site_name  + " (" + sounding_site + "): " + sounding_month + "/" + sounding_day + "/20" + sounding_year + " at " + sounding_hour + " UTC"

# Function to download and parse the data
def download_and_parse(url):
    # Flags to indicate when to start and stop reading
    start_marker = "%RAW%"
    end_marker = "%END%"
    reading_data = False

    # Download the data file
    response = urllib.request.urlopen(url)
    data = response.read().decode('utf-8')

    # Split the data into lines
    lines = data.splitlines()

    # Initialize lists to store columns
    PRESSURE = []
    HGHT = []
    TEMP = []
    DWPT = []
    WDIR = []
    WSPD = []
    print(lines[5])

    # Process each line
    for line in lines:

        if line == start_marker:
            reading_data = True
            continue  # Skip processing the "%RAW%" line itself

        if reading_data:
            if line == end_marker:
                break  # Stop reading if we encounter "%END%"
            else:
                # Split the line into columns
                columns = line.split(',')

                # Convert columns to float where applicable
                level = float(columns[0].strip())
                hght = float(columns[1].strip())
                temp = float(columns[2].strip())
                dwpt = float(columns[3].strip())
                wdir = float(columns[4].strip())
                wspd = float(columns[5].strip())

                # Append to respective lists
                PRESSURE.append(level)
                HGHT.append(hght)
                TEMP.append(temp)
                DWPT.append(dwpt)
                WDIR.append(wdir)
                WSPD.append(wspd)

    # Convert lists to numpy arrays
    PRESSURE = np.array(PRESSURE)
    HGHT = np.array(HGHT)
    TEMP = np.array(TEMP)
    DWPT = np.array(DWPT)
    WDIR = np.array(WDIR)
    WSPD = np.array(WSPD)

    return PRESSURE, HGHT, TEMP, DWPT, WDIR, WSPD

# Download and parse the data
PRESSURE, HGHT, TEMP, DWPT, WDIR, WSPD = download_and_parse(url)

# Print the first few elements of each array as a check
print("PRESSURE:", PRESSURE[:5])
print("HGHT:", HGHT[:5])
print("TEMP:", TEMP[:5])
print("DWPT:", DWPT[:5])
print("WDIR:", WDIR[:5])
print("WSPD:", WSPD[:5])


In [ ]:
# Now we will calculate the relative humidity:
RELHUM = rel_hum(TEMP, DWPT)

# Convert height from meters to kilometers:
HGHT_km = HGHT/1000

In [ ]:
# Other useful functions:

# Constants:
R_d = 287.0 # J/(kg K), gas constant for dry air
R_v = 461.5 # J/(kg K), gas constant for water vapor
eps_vapor = R_d/R_v

# Converting from Celsius to Fahrenheit:
def c2f(celsius_temp):
    fahrenheit_temp = 1.8 * celsius_temp + 32
    return fahrenheit_temp

# Converting from Fahrenheit to Celsius:
def f2c(fahrenheit_temp):
    celsius_temp = (fahrenheit_temp - 32) * (5/9)
    return celsius_temp

# Converting from Celsius to Kelvin:
def c2k(celsius_temp):
    kelvin_temp = celsius_temp + 273.15
    return kelvin_temp

# Converting from Kelvin to Celsius:
def k2c(kelvin_temp):
    celsius_temp = kelvin_temp - 273.15
    return celsius_temp

# Converting from Fahrenheit to Kelvin:
def f2k(fahrenheit_temp):
    kelvin_temp = c2k(f2c(fahrenheit_temp))
    return kelvin_temp

# Converting from Kelvin to Fahrenheit:
def k2f(kelvin_temp):
    fahrenheit_temp = c2f(k2c(kelvin_temp))
    return fahrenheit_temp

# Convert from mixing ratio (kg/kg) to specific humidity (kg/kg):
def w2q(w):
    q = w/(1+w)
    return q

# Convert from specific humidity (kg/kg) to mixing ratio (kg/kg):
def q2w(q):
    w = q/(1-q)
    return w

# Calculate virtual temperature (T_v, K) from temperature (T, K) and specific humidity (q, kg/kg):
def t2tv(T,q):
    # eps_vapor = 287.0/461.5
    T_v = (1 + (1/eps_vapor - 1)*q)*T
    return T_v

# Calculate temperature (T, K) from virtual temperature (T_v, K) and specific humidity (q, kg/kg):
def tv2t(T_v,q):
    T = T_v/(1 + (1/eps_vapor - 1)*q)
    return T

# Calculate vapor pressure (e_vap, Pa) from specific humidity (q, kg/kg) and pressure (p):
def q2e(q,p):
    e_vap = q*p/(eps_vapor - q*eps_vapor + q)
    return e_vap

# Equation of state for dry air or moist air:
def eqstate_air(flag_number,x1,x2):
    if flag_number == 1:
        # solve for p (Pa), assuming x1 = rho (kg/m^3) and x2 = T (K) or T_v (K):
        x3 = R_d*x1*x2
    elif flag_number == 2:
        # solve for T or T_v, assuming x1 = p (Pa) and x2 = rho (kg/m^3):
        x3 = x1/(R_d*x2)
    elif flag_number == 3:
        # solve for rho (kg/m^3), assuming x1 = p (Pa) and x2 = T (K) or T_v (K):
        x3 = x1/(R_d*x2)
    else:
        raise ValueError("the first input, flag_number, must equal 1, 2, or 3")
    return x3

# Equation of state for water vapor:
def eqstate_vapor(flag_number,x1,x2):
    if flag_number == 1:
        # solve for e_vap (Pa), assuming x1 = rho_v (kg/m^3) and x2 = T (K):
        x3 = R_v*x1*x2
    elif flag_number == 2:
        # solve for T (K), assuming x1 = e_vap (Pa) and x2 = rho_v (kg/m^3):
        x3 = x1/(R_v*x2)
    elif flag_number == 3:
        # solve for rho_v (kg/m^3), assuming x1 = e_vap (Pa) and x2 = T (K):
        x3 = x1/(R_v*x2)
    else:
        raise ValueError("the first input, flag_number, must equal 1, 2, or 3")
    return x3

# Supplemental for calculating q from T_v (K) and T (K):
def find_q_from_temps(T,T_v):
    q = (T_v/T - 1)/(1/eps_vapor - 1)
    return q

# Additional unit conversions:
def pa2hpa(p_Pa):
    p_hPa = p_Pa/100
    return p_hPa

def hpa2pa(p_hPa):
    p_Pa = p_hPa*100
    return p_Pa

def g2kg(x_g):
    x_kg = x_g/1000
    return x_kg

def kg2g(x_kg):
    x_g = x_kg*1000
    return x_g

def hpa2inHg(p_hPa): # pressure: hectopascals to inches of mercury
    p_inHg = p_hPa*0.029529983071445
    return p_inHg

def ms2mph(v_ms): # wind: meters per second to miles per hour
    v_mph = v_ms*2.23694
    return v_mph

def mph2ms(v_mph): # wind: miles per hour to meters per second
    v_ms = v_mph/2.23694
    return v_ms

def kt2mph(v_kt): # wind: knots to miles per hour
    v_mph = v_kt*1.15078
    return v_mph

def mph2kt(v_mph): # wind: miles per hour to knots
    v_kt = v_mph/1.15078
    return v_kt

def kt2ms(v_kt): # wind: knots to meters per second
    v_ms = v_kt/1.94384
    return v_ms

def ms2kt(v_ms): # wind: meters per second to knots
    v_kt = v_ms*1.94384
    return v_kt

In [ ]:
plt.rcParams["figure.figsize"] = [13.00, 6.50] # length, width

# HEIGHT/PRESSURE PLOT:
fig_basic_height, axs_basic_height = plt.subplots(1, 3, sharey=True)

# First panel:
colnow = 0
axs_basic_height[colnow].plot(HGHT_km,HGHT_km,'k-',linewidth=1)
axs_basic_height[colnow].set_xlabel('height (km)')
axs_basic_height[colnow].set_ylabel('height (km)')
axs_basic_height[colnow].grid(True)

# pressure axis limit:
min_press = -100 # hPa
max_press = 1070 # hPa

# Second panel:
colnow = 1
axs_basic_height[colnow].plot(PRESSURE,HGHT_km,'r-',linewidth=1)
axs_basic_height[colnow].set_xlabel('Pressure (hPa)')
axs_basic_height[colnow].grid(True)
axs_basic_height[colnow].set_xlim(min_press, max_press)

# Third panel:
colnow = 2
axs_basic_height[colnow].plot(hpa2inHg(PRESSURE),HGHT_km,'r-',linewidth=1)
axs_basic_height[colnow].set_xlabel('Pressure (inches of Hg)')
axs_basic_height[colnow].grid(True)
axs_basic_height[colnow].set_xlim(hpa2inHg(min_press), hpa2inHg(max_press))

fig_basic_height.suptitle(plot_title)

# TEMPERATURE/HUMIDITY PLOT:
fig_basic_temp, axs_basic_temp = plt.subplots(1, 3, sharey=True)

# temperature axis limit:
min_temp = -100 # °C
max_temp = 20 # °C

# First panel:
colnow = 0
axs_basic_temp[colnow].plot(TEMP,HGHT_km,'k-',linewidth=1)
axs_basic_temp[colnow].plot(DWPT,HGHT_km,'b-',linewidth=1)
axs_basic_temp[colnow].legend(['Temperature (°C)','Dew Point (°C)'])
axs_basic_temp[colnow].set_xlabel('degrees Celsius')
axs_basic_temp[colnow].set_ylabel('height (km)')
axs_basic_temp[colnow].set_xlim(min_temp, max_temp)
axs_basic_temp[colnow].grid(True)

# Second panel:
colnow = 1
axs_basic_temp[colnow].plot(c2f(TEMP),HGHT_km,'k-',linewidth=1)
axs_basic_temp[colnow].plot(c2f(DWPT),HGHT_km,'b-',linewidth=1)
axs_basic_temp[colnow].legend(['Temperature (°F)','Dew Point (°F)'])
axs_basic_temp[colnow].set_xlabel('degrees Fahrenheit')
axs_basic_temp[colnow].set_xlim(c2f(min_temp), c2f(max_temp))
axs_basic_temp[colnow].grid(True)

# Third panel:
colnow = 2
axs_basic_temp[colnow].plot(RELHUM,HGHT_km,'-',linewidth=1,color=skyblue_RGB)
axs_basic_temp[colnow].set_xlabel('Relative Humidity (%)')
axs_basic_temp[colnow].set_xlim(-20, 120)
axs_basic_temp[colnow].grid(True)
fig_basic_temp.suptitle(plot_title)

# WIND PLOT:
fig_basic_wind, axs_basic_wind = plt.subplots(1, 4, sharey=True)

# First panel:
colnow = 0
axs_basic_wind[colnow].plot(WDIR,HGHT_km,'m-',linewidth=1)
axs_basic_wind[colnow].set_xlabel('Wind Direction (°)')
axs_basic_wind[colnow].set_ylabel('height (km)')
axs_basic_wind[colnow].grid(True)
axs_basic_wind[colnow].set_xlim(-50, 360)
# Here's how meteorologists define the angles (it's different from what you might have seen in trigonometry):
# https://pressbooks.bccampus.ca/physgeoglabmanual1/wp-content/uploads/sites/1318/2021/03/wind_compass.jpg

# wind speed axis limit:
min_v = -20 # knots
max_v = 200 # knots

# Second panel:
colnow = 1
axs_basic_wind[colnow].plot(WSPD,HGHT_km,'g-',linewidth=1)
axs_basic_wind[colnow].set_xlabel('Wind Speed (knots)')
axs_basic_wind[colnow].grid(True)
axs_basic_wind[colnow].set_xlim(min_v, max_v)

# Third panel:
colnow = 2
axs_basic_wind[colnow].plot(kt2mph(WSPD),HGHT_km,'g-',linewidth=1)
axs_basic_wind[colnow].set_xlabel('Wind Speed (miles per hour)')
axs_basic_wind[colnow].grid(True)
axs_basic_wind[colnow].set_xlim(min_v, max_v) # purposely not converting since knots and mph are quite close
fig_basic_wind.suptitle(plot_title)

# Fourth panel:
colnow = 3
axs_basic_wind[colnow].plot(kt2ms(WSPD),HGHT_km,'g-',linewidth=1)
axs_basic_wind[colnow].set_xlabel('Wind Speed (m/s)')
axs_basic_wind[colnow].grid(True)
axs_basic_wind[colnow].set_xlim(kt2ms(min_v), kt2ms(max_v))
fig_basic_wind.suptitle(plot_title)

plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = [13.00, 6.50] # length, width
fig, axs = plt.subplots(1, 3, sharey=True)

# First panel:
colnow = 0
axs[colnow].plot(TEMP[TEMP != -9999],HGHT_km[TEMP != -9999],'k-',linewidth=1)
axs[colnow].plot(DWPT[DWPT != -9999],HGHT_km[DWPT != -9999],'b-',linewidth=1)
axs[colnow].legend(['Temperature (°C)','Dew Point (°C)'])
axs[colnow].set_xlabel('degrees Celsius')
axs[colnow].set_ylabel('height (km)')
axs[colnow].grid(True)

# Second panel:
colnow = 1
axs[colnow].plot(RELHUM[(TEMP != -9999) & (DWPT != -9999)],HGHT_km[(TEMP != -9999) & (DWPT != -9999)],linewidth=1,color=skyblue_RGB)
axs[colnow].set_xlabel('Relative Humidity (%)')
axs[colnow].grid(True)
axs[colnow].set_xlim(0, 100)

# Third panel:
colnow = 2
axs[colnow].plot(PRESSURE[PRESSURE != -9999],HGHT_km[PRESSURE != -9999],'r-',linewidth=1)
axs[colnow].set_xlabel('Pressure (hPa)')
axs[colnow].grid(True)
axs[colnow].set_xlim(0, 1070)

fig.suptitle(plot_title)
plt.show()



In [ ]:
# Next we will plot data from a severe weather event on January 5, 2025,
# which brought tornadoes, hail, and damaging winds to Arkansas, Louisiana,
# Mississippi, and Alabama.
# More info about this event: https://www.spc.noaa.gov/exper/archive/event.php?date=20250105

# Note: In most cases you will want sounding_hour_2 = "00" or sounding_hour_2 = "12"
sounding_year_2 = "25"
sounding_month_2 = "01"
sounding_day_2 = "05"
sounding_hour_2 = "18"
sounding_site_2 = "SHV"
sounding_site_name_2 = "Shreveport, Louisiana"

url_2 = base_url + sounding_year_2 + sounding_month_2 + sounding_day_2 + sounding_hour_2 + "_OBS/" + sounding_site_2 + ".txt"
plot_title_2 = sounding_site_name_2  + " (" + sounding_site_2 + "): " + sounding_month_2 + "/" + sounding_day_2 + "/20" + sounding_year_2 + " at " + sounding_hour_2 + " UTC"
# Download and parse the data
PRESSURE_2, HGHT_2, TEMP_2, DWPT_2, WDIR_2, WSPD_2 = download_and_parse(url_2)

# Calculate relative humidity:
RELHUM_2 = rel_hum(TEMP_2, DWPT_2)

# Convert height from meters to kilometers:
HGHT_km_2 = HGHT_2/1000

plt.rcParams["figure.figsize"] = [13.00, 6.50] # length, width
fig_2, axs_2 = plt.subplots(1, 3, sharey=True)

# First panel:
colnow = 0
axs_2[colnow].plot(TEMP[TEMP != -9999],HGHT_km[TEMP != -9999],'k:',linewidth=1)
axs_2[colnow].plot(DWPT[DWPT != -9999],HGHT_km[DWPT != -9999],'b:',linewidth=1)
axs_2[colnow].plot(TEMP_2[TEMP_2 != -9999],HGHT_km_2[TEMP_2 != -9999],'k-',linewidth=1)
axs_2[colnow].plot(DWPT_2[DWPT_2 != -9999],HGHT_km_2[DWPT_2 != -9999],'b-',linewidth=1)
axs_2[colnow].legend([('Temperature (°C) @ '+ sounding_site),('Dew Point (°C) @ ' + sounding_site), \
                   ('Temperature (°C) @ '+ sounding_site_2),('Dew Point (°C) @ ' + sounding_site_2)])
axs_2[colnow].set_xlabel('degrees Celsius')
axs_2[colnow].set_ylabel('height (km)')
axs_2[colnow].grid(True)

# Second panel:
colnow = 1
axs_2[colnow].plot(RELHUM[(TEMP != -9999) & (DWPT != -9999)],HGHT_km[(TEMP != -9999) & (DWPT != -9999)],linewidth=1,color=skyblue_RGB,linestyle='dotted')
axs_2[colnow].plot(RELHUM_2[(TEMP_2 != -9999) & (DWPT_2 != -9999)],HGHT_km_2[(TEMP_2 != -9999) & (DWPT_2 != -9999)],linewidth=1,color=skyblue_RGB)
axs_2[colnow].legend([sounding_site,sounding_site_2])
axs_2[colnow].set_xlabel('Relative Humidity (%)')
axs_2[colnow].grid(True)
axs_2[colnow].set_xlim(0, 100)

# Third panel:
colnow = 2
axs_2[colnow].plot(PRESSURE[PRESSURE != -9999],HGHT_km[PRESSURE != -9999],'r:',linewidth=1)
axs_2[colnow].plot(PRESSURE_2[PRESSURE_2 != -9999],HGHT_km_2[PRESSURE_2 != -9999],'r-',linewidth=1)
axs_2[colnow].legend([sounding_site,sounding_site_2])
axs_2[colnow].set_xlabel('Pressure (hPa)')
axs_2[colnow].grid(True)
axs_2[colnow].set_xlim(0, 1070)

fig_2.suptitle(('Comparison: \n ' + plot_title + ', and \n' + plot_title_2))
plt.show()


In [ ]:
# Sounding info: Choose your own!

# List of sites: https://www.weather.gov/upperair/sounding
# (usually need to remove the K at the beginning: KOKX on the map = OKX in the code above)
# (there are some exceptions, such as TJSJ = San Juan, PR; do not remove the T)
# (data from some sites outside the continental US are not available here)
# Another website with similar info: https://www.spc.noaa.gov/exper/soundings/

# Note: In most cases you will want sounding_hour_3 = "00" or sounding_hour_3 = "12"
# This is because most balloon launches occur at 0 UTC and 12 UTC worldwide in order
# to coordinate data collection for weather forecasting models. Sometimes supplemental
# launches occur at other times.

sounding_year_3 = "25"
sounding_month_3 = "01"
sounding_day_3 = "06" # can't go back further than 1 week or so; older data are deleted from this website
sounding_hour_3 = "12"
sounding_site_3 = "GSO"
sounding_site_name_3 = "Greensboro, North Carolina" # the exact format doesn't matter here; this text just goes into the plot title

url_3 = base_url + sounding_year_3 + sounding_month_3 + sounding_day_3 + sounding_hour_3 + "_OBS/" + sounding_site_3 + ".txt"
plot_title_3 = sounding_site_name_3  + " (" + sounding_site_3 + "): " + sounding_month_3 + "/" + sounding_day_3 + "/20" + sounding_year_3 + " at " + sounding_hour_3 + " UTC"
# Download and parse the data
PRESSURE_3, HGHT_3, TEMP_3, DWPT_3, WDIR_3, WSPD_3 = download_and_parse(url_3)

# Calculate relative humidity:
RELHUM_3 = rel_hum(TEMP_3, DWPT_3)

# Convert height from meters to kilometers:
HGHT_km_3 = HGHT_3/1000

# Optional: choose an additional sounding to plot!
# (otherwise the same data from Upton, NY will be reused)
#
# To use this option, change "False" to "True" and then enter the information for
# sounding_year_4, sounding_month_4, etc.
use_new_sounding = False

if use_new_sounding:
    # Remember: In most cases you will want sounding_hour_4 = "00" or sounding_hour_4 = "12"
    sounding_year_4 = ""
    sounding_month_4 = ""
    sounding_day_4 = ""
    sounding_hour_4 = ""
    sounding_site_4 = ""
    sounding_site_name_4 = ""

    url_4 = base_url + sounding_year_4 + sounding_month_4 + sounding_day_4 + sounding_hour_4 + "_OBS/" + sounding_site_4 + ".txt"
    plot_title_4 = sounding_site_name_4  + " (" + sounding_site_4 + "): " + sounding_month_4 + "/" + sounding_day_4 + "/20" + sounding_year_4 + " at " + sounding_hour_4 + " UTC"

    # Download and parse the data
    PRESSURE_4, HGHT_4, TEMP_4, DWPT_4, WDIR_4, WSPD_4 = download_and_parse(url_4)

    # Calculate relative humidity:
    RELHUM_4 = rel_hum(TEMP_4, DWPT_4)

    # Convert height from meters to kilometers:
    HGHT_km_4 = HGHT_4/1000
else:
    # If we are not adding an additional sounding, then we will use the
    # original sounding we plotted from Upton, NY.
    PRESSURE_4 = PRESSURE
    HGHT_4 = HGHT
    TEMP_4 = TEMP
    DWPT_4 = DWPT
    WDIR_4 = WDIR
    WSPD_4 = WSPD
    RELHUM_4 = RELHUM
    HGHT_km_4 = HGHT_km
    plot_title_4 = plot_title
    sounding_site_4 = sounding_site
    sounding_site_name_4 = sounding_site_name

plt.rcParams["figure.figsize"] = [13.00, 6.50] # length, width
fig_3, axs_3 = plt.subplots(1, 3, sharey=True)

# First panel:
colnow = 0
axs_3[colnow].plot(TEMP_3[TEMP_3 != -9999],HGHT_km_3[TEMP_3 != -9999],'k-',linewidth=1)
axs_3[colnow].plot(DWPT_3[DWPT_3 != -9999],HGHT_km_3[DWPT_3 != -9999],'b-',linewidth=1)
axs_3[colnow].plot(TEMP_4[TEMP_4 != -9999],HGHT_km_4[TEMP_4 != -9999],'k:',linewidth=1)
axs_3[colnow].plot(DWPT_4[DWPT_4 != -9999],HGHT_km_4[DWPT_4 != -9999],'b:',linewidth=1)
axs_3[colnow].legend([('Temperature (°C) @ '+ sounding_site_3),('Dew Point (°C) @ ' + sounding_site_3), \
                   ('Temperature (°C) @ '+ sounding_site_4),('Dew Point (°C) @ ' + sounding_site_4)])
axs_3[colnow].set_xlabel('degrees Celsius')
axs_3[colnow].set_ylabel('height (km)')
axs_3[colnow].grid(True)

# Second panel:
colnow = 1
axs_3[colnow].plot(RELHUM_3[(TEMP_3 != -9999) & (DWPT_3 != -9999)],HGHT_km_3[(TEMP_3 != -9999) & (DWPT_3 != -9999)],linewidth=1,color=skyblue_RGB)
axs_3[colnow].plot(RELHUM_4[(TEMP_4 != -9999) & (DWPT_4 != -9999)],HGHT_km_4[(TEMP_4 != -9999) & (DWPT_4 != -9999)],linewidth=1,color=skyblue_RGB,linestyle='dotted')
axs_3[colnow].legend([sounding_site_3,sounding_site_4])
axs_3[colnow].set_xlabel('Relative Humidity (%)')
axs_3[colnow].grid(True)
axs_3[colnow].set_xlim(0, 100)

# Third panel:
colnow = 2
axs_3[colnow].plot(PRESSURE_3[PRESSURE_3 != -9999],HGHT_km_3[PRESSURE_3 != -9999],'r-',linewidth=1)
axs_3[colnow].plot(PRESSURE_4[PRESSURE_4 != -9999],HGHT_km_4[PRESSURE_4 != -9999],'r:',linewidth=1)
axs_3[colnow].legend([sounding_site_3,sounding_site_4])
axs_3[colnow].set_xlabel('Pressure (hPa)')
axs_3[colnow].grid(True)
axs_3[colnow].set_xlim(0, 1070)

fig_3.suptitle(('Comparison: \n ' + plot_title_3 + ', and \n' + plot_title_4))
plt.show()